## About 'Robot Localization with Particle Filters'

This project an application of Particle Filters.

What is Robot Localization?

- It is the process of determining where a mobile robot is located w.r.t its environment.
- Localization is one of the fundamental competencies required by an autonomous robot as the knowledge of the robot's own location is an essential precursor to making decisions about future actions

The problem statement I tried to solve here is pretty much the same: Simulating a robot that can move around in an unknown environment, and have it discover its own location using only a terrain map and an Altimeter, i.e an elevation sensor.

## Importing necessary libraries

In [ ]:
#Used for working with arrays
import numpy as np
#Used for performing image processing and computer vision tasks
import cv2

## Loading the terrain map

In [ ]:
#Loading the map as a grayscale image (hence '0' as an argument)
map = cv2.imread("map.png", 0)
HEIGHT, WIDTH = map.shape

#Printing the map as a 2D-array which contains pixel brightness values (0-black to 255-white).
#These pixel values will be used as the terrain map itself. These pixel values will be taken as the terrain height in metres.
print(map)

#Initializing the robot's position somewhere on the map using x, y and z axes co-ordinates
rx, ry, rtheta = (WIDTH/4, HEIGHT/4, 0)

[[ 79  80  82 ... 133 148 156]
 [ 78  79  81 ... 138 156 164]
 [ 76  77  79 ... 147 170 180]
 ...
 [181 181 182 ... 174 172 171]
 [178 178 178 ... 180 179 178]
 [177 177 177 ... 183 182 182]]


## Reading keyboard input to move the robot

In [ ]:
#Assigning 5 pixels movement straight in response to the keyboard input
STEP = 5
#Assigning 25 degrees (converted to radians) movement in the left or right direction in response to the keyboard input
TURN = np.radians(25)

def get_input():

    #Initializing everything to 0
    fwd = 0
    turn = 0
    halt = False

    #This captures keyboard input
    #0: wait indefinitely for the keyboard input
    k = cv2.waitKey(0)

    #If input is the up arrow key (82 is the ASCII code)
    if k == 82:
        fwd = STEP
    #If input is the right arrow
    elif k == 83:
        turn = TURN
    #If the input is the left arrow
    elif k == 81:
        turn = -TURN
    #Halt the program if the input is anything else
    else:
        halt = True

    return fwd, turn, halt

## Moving the robot with Gaussian noise

In the real-world, the terrain is uneven and the robot control is not perfect. So, there's some uncertainty about how the robot will move after taking our commands.

We must take this into consideration while coding the algorithm. And we can simulate this by adding Gaussian noise to the stepping and turning values, i.e. assuming that there will be some deviation in the robot's actual movement vs its predicted movement (called Standard Deviation)

In [ ]:
#Assigning SD in forward movement to half a pixel
SIGMA_STEP = 0.5
#Assigning SD in turning left or right to 5 degrees (converted to radians)
SIGMA_TURN = np.radians(5)

#Moving the robot
def move_robot(rx, ry, rtheta, fwd, turn):

    #This is the noisy forward distance, i.e. the real-world forward distance of the robot.
    #It gives a Normal data distribution.
    #fwd: Mean, i.e. the fwd value we expect the robot to move
    #SIGMA_STEP: value of the SD
    #1: number of values to be returned
    fwd_noisy = np.random.normal(fwd, SIGMA_STEP, 1)

    #Incrementing the robot's x-coordinate by the noisy forwad distance * cosine(heading angle)
    rx += fwd_noisy * np.cos(rtheta)
    #Incrementing the robot's y-coordinate by the noisy forwad distance * sine(heading angle)
    ry += fwd_noisy * np.sin(rtheta)

    turn_noisy = np.random.normal(turn, SIGMA_TURN, 1)
    #Incrementing the robot's z-coordinate by the noisy turning command
    rtheta += turn_noisy

    return rx, ry, rtheta

## Initializing a Particle Filter

What is a Particle Filter?
- It is a Bayesian filter, i.e. estimation is performed using Bayesian theory. This allows for estimating a state by combining a statistical model for a measurement (likelihood) with prior probability using Bayes' theorem.
- It keeps track of thousands of different position and orientation estimates (the particles), and each one is weighted according to how well it fits the measurements from the sensor.


How to implement this concept in the project?
- Firstly, we place a few thousand particles all over the terrain map at random positions and random orientations.
- Each particle represents a hypothesis about where the robot might be.
- When we give a command to the robot, we will move the particles in the same way as well. And then we try to improve our cloud of particles, and obtain an estimate that converges to the robot location.

In [ ]:
#Placing 3000 particles randomly on the terrain map
NUM_PARTICLES = 3000

#Keeping track of the particles
def init():

    #Setting all particle values to random numbers between 0 and 1
    #NUM_PARTICLES: number of rows
    #3: number of columns (one each for x, y, z(theta))
    particles = np.random.rand(NUM_PARTICLES, 3)
    #Spreading out the particles all over the map by scaling up the 'particles' array.
    #Scaling up: multiplying the 'particles' array with another array containing the max values of the width, height and theta
    particles *= np.array( (WIDTH, HEIGHT, np.radians(360)))

    return particles

# RED CIRCLE IS THE BEST GUESS. IT IS THE AVERAGE OF ALL THE PARTICLES

## Moving the particles around everytime a keyboard command is given

In [ ]:
#Moving the particles one column at a time
def move_particles(particles, fwd, turn):

    #Multiplying each column by the cosine or sine of the heading (where the robot's headed) of that particular particle
    #[:,0]: selecting all rows and the 0th (first) coloumn. This is the x-coordinate
    particles[:,0] += fwd * np.cos(particles[:,2])
    #[:,1]: selecting all rows and the 1th (second) coloumn. This is the y-coordinate
    particles[:,1] += fwd * np.sin(particles[:,2])
    #[:,2]: selecting all rows and the 2th (third) coloumn. This is the z-coordinate
    particles[:,2] += turn

    #Preventing particles from falling off the edge of the map
    #0.0: lower bound
    #WIDTH-1: upper bound
    particles[:,0] = np.clip(particles[:,0], 0.0, WIDTH-1)
    particles[:,1] = np.clip(particles[:,1], 0.0, HEIGHT-1)

    return particles

## Getting Input from the robot's Elevation sensor

In the real-world, the sensor's input will contain some noise. And this will be taken into consideration by adding Gaussian noise.

In [ ]:
#Assigning SD of 2 pixels in the sensor's input
SIGMA_SENSOR = 2

def sense(x, y, noisy=False):

    #Casting x and y to integers as we want to use them as indexes for the array
    x = int(x)
    y = int(y)

    #The robot doesn't know its location, but we are running the simulation. We do. Just look up the elevation on the map.
    if noisy:

        #map(y,x): Mean, i.e. the actual elevation. It is indexed as (y,x) because that's the way images are stored.
        #SIGMA_Sensor: value of the SD
        #1: number of values to be returned
        return np.random.normal(map(y,x), SIGMA_SENSOR, 1)

    #Returns the elevation under the particles
    return map[y,x]

## Assigning and Computing particle weights

A higher weight means that the elevation is a closer match to the robot's sensor measurement.

In [ ]:
#Computing particle weights
def compute_weights(particles, robot_sensor):

    #Initializing an array containing 3000 '0's
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):

        #Getting each particle's elevation using the 'sense' function
        #particles[i,0]: x-coordinate of the particle
        #particles[i,1]: y-coordinate of the particle
        #noisy=False: noise-free
        elevation = sense(particles[i,0], particles[i,1], noisy=False)

        #Storing robot's sensor measurement - particle elevation in the 'errors' array.
        #This signifies how far off the particle is from what the robot thinks the elevation is.
        #We take the absolute value because we're just interested in the closeness.
        errors[i] = abs(robot_sensor - elevation)

    #Assigning weights to the particles.
    #Weight must go up when the error goes down.
    #This ensures that we'll have one weight for each error, but it will move in the opposite direction.
    weights = np.max(error) - errors

    #Setting the particle weight to 0 when a particle reaches the edge, as they shouldn't pile up on the edge of the map.
    weights[
        (particles[:,0] == 0) |
        (particles[:,0] == WIDTH-1) |
        (particles[:,1] == 0) |
        (particles[:,1] == HEIGHT-1)
    ] = 0.0

    #Cubing the weights to increase the sensitivity of the weight values to the terrain elevation.
    #So, for a small difference between the robot and the particles, we'll see more difference in the weights.
    #This makes the particle filter work better.
    weights = weights ** 3

    return weights

## Resampling the particles

In [ ]:
#Resampling the particle weights to improve our estimate
def resample(particles, weights):

    #Normalizing the weights so that they sum up to 1.
    #This allows us to use them as a probabilty distribution over the particles.
    probabilities = weights / np.sum(weights)

    #Sampling/Randomizing the particles based on the above probability distribution.
    #Particles with high weights get resampled many times, and those with low weights may not have been resampled at all.
    #And this improves the estimate of the robot.
    #NUM_PARTICLES: set of objects to sample from
    #size = NUM_PARTICLES: number of values to return
    #p = probabilities: randomizing on the basis of this probability distribution
    #This returns an array of row index number
    new_index = np.random.choice(
        NUM_PARTICLES,
        size = NUM_PARTICLES,
        p = probabilities
    )

    #Creating a new particle array from the randomized indices obtained from above
    particles = particles[new_index,:]

    return particles

## Adding Noise to the particles

There's a lot of uncertainty about what the robot's movement.
There's noise in its movement and steering, there's noise in the elevation sensor's measurement.

So, we'll add some random variations to the particles. And it's likely that some of the particles will have variations that match the robot.

In [ ]:
#Assigning SD of 2 pixels to each particle's position
SIGMA_POS = 2
#Assigning SD of 10 degrees (converted to radians) to the orientation, i.e. z-coordinate
SIGMA_TURN = np.radians(10)

#Constructing and Adding noise to the particles
def add_noise(particles):

    #Constructing noise by column-wise (axis=1) concatenation
    noise = np.concatenate(
        (
            np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
            np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
            np.random.normal(0, SIGMA_TURN, (NUM_PARTICLES, 1)),
        ),
        axis=1
    )

    #Adding the constructed noise to the particles
    particles += noise

    return particles

## Visualizing the robot, particles and best guess

In [ ]:
#Visualizing the project components
def display(map, rx, ry, particles):

    #Converts an image from one color space to another.
    #cv2.COLOR_GRAY2BGR: replaces all B, G, R channels with gray value
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)

    #Visualizing the particles
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):

            #Drawing a circle on 'lmap'
            #lmap: image on which the circle needs to be drawn
            #(int(particles[i,0]), int(particles[i,1])): centre co-ordinates
            #1: radius
            #(255,0,0): color
            #1: thickness
            cv2.circle(lmap,
                       (int(particles[i,0]), int(particles[i,1])),
                       1,
                       (255,0,0),
                       1)

    #Visualizing the robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)

    #Visualizing the best guess
    if len(particles) > 0:

        #x-coordinate of the mean of all particles
        px = np.mean(particles[:,0])
        #y-coordinate of the mean of all particles
        py = np.mean(particles[:,1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0,0,255), 5)

    cv2.imshow('map', lmap)

## Main routine

In [ ]:
particles = init()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)

cv2.destroyAllWindows()


## Sample visuals

Legend for the below visuals:
- Green Circle: Robot
- Small Blue Dots: Particles
- Red Circle: Best Estimate

Working of the algorithm:
- At first, all visuals come up and are static (Image-1)
- Once we start using the arrow keys to move and steer the robot, the particles start coming together gradually (Image-2)
- As we move and steer the robot more, the particles start concentrating in smaller and smaller regions (Image-3)
- Eventually, all particles lock onto the robot, and the Red Circle, which is the Best Estimate (cumulative average of all the particles) completely overlaps with the Green Circle (robot), and follows it wherever it moves.

![Image-1](attachment:sample1.PNG)

![Image-2](attachment:sample2.PNG)

![Image-3](attachment:sample3.PNG)

![Image-4](attachment:image.png)